In [10]:
# Cell 1: Environment and Imports
import os
import sys
from dotenv import load_dotenv
# Import our LSTM components
from comp5.config.settings import LSTMConfig, AstraDBConfig
from comp5.core.gate_networks import LSTMGateNetwork
from comp5.core.memory_manager import MemoryManager
from comp5.models.memory_item import MemoryItem
from comp5.models.rl_experience import RLExperience, RLState, RLAction, RLReward
from comp5.database.astra_connector import AstraDBConnector
from comp5.rl_training.gate_optimizer import GateOptimizer
from comp5.rl_training.reward_calculator import RewardCalculator
from comp5.utils.similarity import cosine_similarity
from comp5.utils.token_counter import estimate_tokens

NameError: name 'DATABASE_ID' is not defined

## Component 5 gets data from Astra DB

In [ ]:
import os
import json
from uuid import UUID
from dotenv import load_dotenv
from astrapy import DataAPIClient

# Load env vars
load_dotenv()

user_id_fs = "123"
API_ENDPOINT = os.getenv("ASTRA_DB_API_ENDPOINT")
API_TOKEN = os.getenv("ASTRA_DB_TOKEN")
KEYSPACE = os.getenv("KEYSPACE", "default_keyspace")
COLLECTION = os.getenv("COLLECTION", "chat_embeddings")   # or "memory_embeddings" if that's created

# Connect client
client = DataAPIClient(API_TOKEN)
db = client.get_database_by_api_endpoint(API_ENDPOINT)
collection = db.get_collection(COLLECTION, keyspace=KEYSPACE)

# Query memory embeddings for user_id "123"
# Modify the query syntax if needed, depending on how your records are stored
query = {"user_id": user_id_fs}  # Filter records

docs = collection.find(query)

formatted_results = []

for doc in docs:
    # Remap/transform as needed for the target schema
    result = {
        "id": str(doc.get('id', '')),  # Make sure this is UUID format in your DB
        "user_id": doc.get('user_id', ''),
        "memory_type": doc.get('memory_type', ''),
        "content_summary": doc.get('content_summary', ''),
        "importance_score": doc.get('importance_score', 0.0),
        "gate_scores": json.dumps(doc.get('gate_scores', {})),  # ensure JSON string
        "feature_vector": doc.get('feature_vector', []),
        "created_at": doc.get('created_at', ''),
        "last_accessed": doc.get('last_accessed', ''),
        "access_frequency": doc.get('access_frequency', 0),
        "emotional_significance": doc.get('emotional_significance', 0.0),
        "temporal_relevance": doc.get('temporal_relevance', 0.0),
        "relationships": doc.get('relationships', []),
        "context_needed": json.dumps(doc.get('context_needed', {})),
        "retrieval_triggers": doc.get('retrieval_triggers', []),
        "original_entry_id": str(doc.get('original_entry_id', '')),
    }
    formatted_results.append(result)


In [12]:
# Cell 2: Initialize LSTM System
async def initialize_lstm_system():
    """Initialize the complete LSTM memory system"""
    
    # Create configuration
    config = LSTMConfig()
    config.astra_db.endpoint = API_ENDPOINT
    config.astra_db.token = API_TOKEN
    config.astra_db.keyspace = KEYSPACE
    
    # Initialize memory manager
    memory_manager = MemoryManager(config)
    await memory_manager.initialize()
    
    # Initialize RL trainer
    rl_trainer = GateOptimizer(
        gate_network=memory_manager.gate_network,
        learning_rate=0.001
    )
    
    return memory_manager, rl_trainer, config

# Run initialization
memory_manager, rl_trainer, config = await initialize_lstm_system()
print("✅ LSTM system initialized successfully")
print(f"📊 Gate thresholds: {memory_manager.gate_network.get_thresholds()}")

2025-09-03 05:39:55,093 - comp5.core.memory_manager - INFO - Initializing Memory Manager...
2025-09-03 05:39:55,093 - comp5.database.astra_connector - INFO - Connecting to Astra DB: https://a240df36-45c0-4e1a-a227-2c9a3b1b4822-us-east1.apps.astra.datastax.com
2025-09-03 05:39:55,286 - astrapy.data.database - INFO - findCollections
2025-09-03 05:39:56,457 - astrapy.data.database - INFO - finished findCollections
2025-09-03 05:39:56,457 - comp5.database.astra_connector - INFO - Connected! Found collections: ['chat_embeddings', 'vector_embeddings']
2025-09-03 05:39:57,638 - comp5.database.astra_connector - INFO - Successfully connected to Astra DB
2025-09-03 05:39:57,648 - comp5.core.memory_manager - INFO - No existing gate network found, using fresh model
2025-09-03 05:39:57,648 - comp5.core.memory_manager - INFO - Memory Manager initialized successfully


✅ LSTM system initialized successfully
📊 Gate thresholds: {'forget': 0.30000001192092896, 'input': 0.4000000059604645, 'output': 0.4000000059604645}


In [20]:
import random
# Sample journal entries
sample_entries = formatted_results

# Process entries through LSTM system
print("📝 Processing journal entries through LSTM gates...")
created_memories = []

for i, entry in enumerate(sample_entries, 1):
    print(entry)
    # Generate dummy feature vector (replace with Component 4)
    feature_vector = entry['feature_vector']
    
    # Generate dummy embeddings (replace with Component 3)  
    embeddings = [random.random() for _ in range(768)]
    
    # Process through memory manager
    memory = await memory_manager.process_new_entry(
        user_id='123',
        feature_vector=feature_vector,
        content=entry,
        embeddings=embeddings
    )
    
    if memory:
        created_memories.append(memory)
        print(f"✅ Memory created: {memory.memory_type} (importance: {memory.importance_score:.3f})")
    else:
        print("❌ Entry not stored (low importance)")

print(f"\n📊 Results: {len(created_memories)}/{len(sample_entries)} entries became memories")

2025-09-03 05:57:36,590 - comp5.core.memory_manager - INFO - Processing new entry for user 123
2025-09-03 05:57:36,591 - comp5.core.memory_manager - INFO - Input gate score: 0.646, decision: True
2025-09-03 05:57:36,592 - comp5.core.memory_manager - ERROR - Error processing new entry for user 123: 'dict' object has no attribute 'lower'
2025-09-03 05:57:36,592 - comp5.core.memory_manager - INFO - Processing new entry for user 123
2025-09-03 05:57:36,593 - comp5.core.memory_manager - INFO - Input gate score: 1.000, decision: True
2025-09-03 05:57:36,595 - comp5.core.memory_manager - ERROR - Error processing new entry for user 123: 'dict' object has no attribute 'lower'
2025-09-03 05:57:36,595 - comp5.core.memory_manager - INFO - Processing new entry for user 123
2025-09-03 05:57:36,597 - comp5.core.memory_manager - INFO - Input gate score: 0.608, decision: True
2025-09-03 05:57:36,598 - comp5.core.memory_manager - ERROR - Error processing new entry for user 123: 'dict' object has no attr

📝 Processing journal entries through LSTM gates...
{'id': 'bfd617c0-8ca4-4c9c-ada3-09ff95ab3819', 'user_id': '123', 'memory_type': 'conversation', 'content_summary': 'I had my English exam today. I have a date with Sarah tomorrow', 'importance_score': 0.3504100122237817, 'gate_scores': '{"forget_score": 0.5, "input_score": 0.35589662194252014, "output_score": 0.32, "confidence": 0.7039125472307205}', 'feature_vector': [0.370590478181839, 0.9829629063606262, 0.5, 1, 0, 0.5, 1, 1, 0.75, 0.699999988079071, 0.5249999761581421, 0.5, 0.5, 0.9791666269302368, 0.5, 1, 0.5, 0.5, 0.5, 0.5, 0.5714285969734192, 1, 0.75, 0.699999988079071, 0.5, 0.3630714416503906, 0.7072107195854187, 0, 0.6042777895927429, 0.9751241207122803, 0.01113926898688078, 0.505569577217102, 0.5000373721122742, 1, 0.01113926898688078, 0.01113926898688078, 0.01113926898688078, 0.01113926898688078, 0.01113926898688078, 0.01113926898688078, 0.01113926898688078, 0.2222352772951126, 0.02227853797376156, 0.6897003054618835, 0.3298

In [21]:
# Cell 4: Test Context Retrieval
print("🔍 Testing memory context retrieval...")

# Test queries
test_queries = [
    "How's work going?",
    "Tell me about AI",
    "Any upcoming appointments?",
    "How am I feeling lately?"
]

for query in test_queries:
    print(f"\n❓ Query: '{query}'")
    
    # Generate query features (placeholder)
    query_features = [random.random() for _ in range(90)]
    
    # Get relevant context
    relevant_memories, metadata = await memory_manager.get_relevant_context(
        user_id="123",
        query=query,
        query_features=query_features,
        max_tokens=1500
    )
    
    print(f"🎯 Found {len(relevant_memories)} relevant memories")
    print(f"📊 Metadata: {metadata}")
    
    if relevant_memories:
        for memory in relevant_memories[:2]:  # Show top 2
            print(f"   💭 {memory.memory_type}: {memory.content_summary[:60]}...")
            print(f"      Importance: {memory.importance_score:.3f}")

2025-09-03 06:00:20,589 - comp5.core.memory_manager - INFO - Assembling context for user 123
2025-09-03 06:00:20,590 - astrapy.data.cursors.cursor - INFO - cursor fetching a page: (empty page state) from memory_embeddings


🔍 Testing memory context retrieval...

❓ Query: 'How's work going?'


2025-09-03 06:00:21,770 - astrapy.utils.api_commander - WARNING - APICommander about to raise from: [{'message': 'Collection does not exist, collection name: memory_embeddings', 'errorCode': 'COLLECTION_NOT_EXIST', 'id': '9ae76c7a-8292-4c86-87bc-2e1d28c54f78', 'family': 'REQUEST', 'title': 'Collection does not exist, collection name', 'scope': 'EMPTY'}]
2025-09-03 06:00:21,771 - comp5.database.astra_connector - ERROR - Error getting user memories: Collection does not exist, collection name: Collection does not exist, collection name: memory_embeddings (COLLECTION_NOT_EXIST)
2025-09-03 06:00:21,771 - comp5.database.memory_store - INFO - Retrieved 0 memories for user 123
2025-09-03 06:00:21,772 - comp5.core.memory_manager - INFO - No memories found for user 123
2025-09-03 06:00:21,772 - comp5.core.memory_manager - INFO - Assembling context for user 123
2025-09-03 06:00:21,773 - comp5.core.memory_manager - INFO - No memories found for user 123
2025-09-03 06:00:21,773 - comp5.core.memory_m

🎯 Found 0 relevant memories
📊 Metadata: {'reason': 'no_memories_found'}

❓ Query: 'Tell me about AI'
🎯 Found 0 relevant memories
📊 Metadata: {'reason': 'no_memories_found'}

❓ Query: 'Any upcoming appointments?'
🎯 Found 0 relevant memories
📊 Metadata: {'reason': 'no_memories_found'}

❓ Query: 'How am I feeling lately?'
🎯 Found 0 relevant memories
📊 Metadata: {'reason': 'no_memories_found'}


In [ ]:
# Cell 5: RL Training Setup
print("🎓 Setting up RL training...")

# Create reward calculator
reward_calc = RewardCalculator()

# Simulate user feedback for training
def simulate_user_interaction(memories_used: List[MemoryItem]) -> Dict:
    """Simulate user interaction data"""
    return {
        'response_length': random.randint(50, 200),
        'has_follow_up_questions': random.choice([True, False]),
        'session_duration_minutes': random.uniform(2, 15),
        'positive_feedback': random.choice([True, False]),
        'context_relevance_score': random.uniform(0.3, 1.0),
        'memories_used': len(memories_used),
        'processing_time_seconds': random.uniform(0.5, 2.0)
    }

# Test reward calculation
print("🏆 Testing reward calculation...")

for i in range(3):
    # Simulate interaction
    interaction_data = simulate_user_interaction(created_memories[:3])
    
    # Calculate reward using our reward calculator
    reward = reward_calc.calculate_simple_reward({
        'user_engagement': random.uniform(0.4, 1.0),
        'context_relevance': random.uniform(0.3, 1.0), 
        'memory_efficiency': random.uniform(0.5, 1.0),
        'user_satisfaction': random.uniform(0.4, 1.0),
        'conversation_quality': random.uniform(0.5, 1.0)
    })
    
    print(f"🎯 Interaction {i+1}: Reward = {reward:.3f}")
    print(f"   Engagement metrics: {interaction_data}")

print("✅ Reward calculation working")

## Component 5 (LSTM)

In [ ]:
# #!/usr/bin/env python3
# """
# Component 5 Pipeline - LSTM Memory Gates Processing
# ==================================================

# This script processes retrieved Astra DB data through Component 5's memory gates system:
# 1. Retrieves memories from Astra DB for user_id="123"
# 2. Processes through LSTM gates (Input, Forget, Output)
# 3. Applies intelligent memory filtering and scoring
# 4. Outputs processed memories ready for Component 6

# Architecture:
# Astra DB → Feature Processing → LSTM Gates → Memory Filtering → Component 6 Input
# """

# import os
# import sys
# import json
# import asyncio
# import numpy as np
# from typing import List, Dict, Any, Optional, Tuple
# from datetime import datetime, timedelta
# from uuid import UUID
# from dotenv import load_dotenv
# from astrapy import DataAPIClient

# # Add comp5 to Python path
# project_root = os.path.dirname(os.path.abspath(__file__))
# comp5_path = os.path.join(project_root, 'comp5')
# sys.path.insert(0, comp5_path)

# # Component 5 imports (simplified for pipeline)
# from comp5_lstm_config import LSTMConfig
# from simple_lstm_gates import SimpleLSTMGates

# # Load environment variables
# load_dotenv()

# class Component5Pipeline:
#     """
#     Complete Component 5 processing pipeline
#     Handles memory retrieval, gate processing, and intelligent filtering
#     """
    
#     def __init__(self):
#         self.config = self._create_config()
#         self.gate_network = None
#         self.memory_manager = None
#         self.astra_client = None
#         self.collection = None
        
#         # Processing statistics
#         self.stats = {
#             'memories_retrieved': 0,
#             'memories_processed': 0,
#             'input_gate_accepted': 0,
#             'forget_gate_triggered': 0,
#             'output_gate_filtered': 0,
#             'final_memories_count': 0,
#             'processing_time': 0.0
#         }
    
#     def _create_config(self) -> LSTMConfig:
#         """Create Component 5 configuration from environment"""
        
#         # Astra DB config
#         astra_config = AstraDBConfig(
#             endpoint=os.getenv("ASTRA_DB_API_ENDPOINT", ""),
#             token=os.getenv("ASTRA_DB_TOKEN", ""),
#             keyspace=os.getenv("KEYSPACE", "default_keyspace")
#         )
        
#         # Gate network config
#         gate_config = GateNetworkConfig(
#             input_size=90,  # Feature vector size from Component 4
#             hidden_size=128,
#             dropout_rate=0.1
#         )
        
#         # Memory config
#         memory_config = MemoryConfig(
#             forget_threshold=0.3,
#             input_threshold=0.4,  # 40% acceptance rate
#             output_threshold=0.4,
#             max_context_tokens=2000,
#             max_memories_per_context=20
#         )
        
#         return LSTMConfig(
#             astra_db=astra_config,
#             gate_network=gate_config,
#             memory=memory_config
#         )
    
#     async def initialize(self):
#         """Initialize the Component 5 pipeline"""
#         print("🚀 Initializing Component 5 Pipeline...")
        
#         # Initialize Astra DB connection
#         await self._init_astra_connection()
        
#         # Initialize gate networks
#         self.gate_network = LSTMGateNetwork(
#             input_size=self.config.gate_network.input_size,
#             hidden_size=self.config.gate_network.hidden_size,
#             dropout=self.config.gate_network.dropout_rate
#         )
        
#         # Try to load pre-trained model
#         try:
#             model_path = os.path.join(comp5_path, "models", "gate_network.pt")
#             if os.path.exists(model_path):
#                 self.gate_network = LSTMGateNetwork.load_model(model_path)
#                 print("✅ Loaded pre-trained gate network")
#             else:
#                 print("📝 Using fresh gate network (no pre-trained model found)")
#         except Exception as e:
#             print(f"⚠️ Could not load pre-trained model: {e}")
        
#         # Initialize memory manager
#         self.memory_manager = MemoryManager(self.config)
        
#         print("✅ Component 5 Pipeline initialized successfully")
    
#     async def _init_astra_connection(self):
#         """Initialize Astra DB connection"""
#         api_endpoint = os.getenv("ASTRA_DB_API_ENDPOINT")
#         api_token = os.getenv("ASTRA_DB_TOKEN")
#         keyspace = os.getenv("KEYSPACE", "default_keyspace")
#         collection_name = os.getenv("COLLECTION", "chat_embeddings")
        
#         if not api_endpoint or not api_token:
#             raise ValueError("Missing ASTRA_DB_API_ENDPOINT or ASTRA_DB_TOKEN")
        
#         # Connect to Astra DB
#         self.astra_client = DataAPIClient(api_token)
#         db = self.astra_client.get_database_by_api_endpoint(api_endpoint)
#         self.collection = db.get_collection(collection_name, keyspace=keyspace)
        
#         print(f"✅ Connected to Astra DB: {collection_name}")
    
#     async def retrieve_user_memories(self, user_id: str) -> List[Dict[str, Any]]:
#         """Retrieve memories from Astra DB for the specified user"""
#         print(f"📊 Retrieving memories for user: {user_id}")
        
#         try:
#             # Query Astra DB
#             query = {"user_id": user_id}
#             docs = list(self.collection.find(query))
            
#             formatted_memories = []
#             for doc in docs:
#                 # Convert Astra DB document to Component 5 format
#                 memory = {
#                     "id": str(doc.get('_id', '')),
#                     "user_id": doc.get('user_id', ''),
#                     "memory_type": doc.get('memory_type', 'conversation'),
#                     "content_summary": doc.get('content_summary', ''),
#                     "importance_score": float(doc.get('importance_score', 0.0)),
#                     "gate_scores": doc.get('gate_scores', {}),
#                     "feature_vector": doc.get('feature_vector', []),
#                     "created_at": doc.get('created_at', ''),
#                     "last_accessed": doc.get('last_accessed', ''),
#                     "access_frequency": int(doc.get('access_frequency', 0)),
#                     "emotional_significance": float(doc.get('emotional_significance', 0.0)),
#                     "temporal_relevance": float(doc.get('temporal_relevance', 0.0)),
#                     "relationships": doc.get('relationships', []),
#                     "context_needed": doc.get('context_needed', {}),
#                     "retrieval_triggers": doc.get('retrieval_triggers', []),
#                     "original_entry_id": str(doc.get('original_entry_id', ''))
#                 }
#                 formatted_memories.append(memory)
            
#             self.stats['memories_retrieved'] = len(formatted_memories)
#             print(f"✅ Retrieved {len(formatted_memories)} memories from Astra DB")
#             return formatted_memories
            
#         except Exception as e:
#             print(f"❌ Error retrieving memories: {e}")
#             return []
    
#     def _ensure_feature_vector(self, memory: Dict[str, Any]) -> List[float]:
#         """Ensure memory has a valid 90-dimension feature vector"""
#         feature_vector = memory.get('feature_vector', [])
        
#         if not feature_vector or len(feature_vector) != 90:
#             # Generate synthetic feature vector if missing
#             # In production, this would come from Component 4
#             print(f"⚠️ Generating synthetic feature vector for memory {memory.get('id', 'unknown')}")
            
#             # Create features based on available metadata
#             importance = memory.get('importance_score', 0.5)
#             emotional = memory.get('emotional_significance', 0.5)
#             temporal = memory.get('temporal_relevance', 0.5)
#             access_freq = min(memory.get('access_frequency', 0) / 10.0, 1.0)
            
#             # Memory type encoding (one-hot)
#             memory_type = memory.get('memory_type', 'conversation')
#             type_encoding = [0.0] * 4
#             type_map = {'conversation': 0, 'event': 1, 'emotion': 2, 'insight': 3}
#             if memory_type in type_map:
#                 type_encoding[type_map[memory_type]] = 1.0
            
#             # Content features (synthetic)
#             content_length = len(memory.get('content_summary', '')) / 500.0
#             content_features = [content_length] * 10
            
#             # Temporal features
#             try:
#                 created_at = datetime.fromisoformat(memory.get('created_at', '2024-01-01'))
#                 days_old = (datetime.now() - created_at).days / 365.0  # Normalize to years
#             except:
#                 days_old = 0.5
            
#             temporal_features = [days_old] * 5
            
#             # Relationship features
#             relationship_count = len(memory.get('relationships', [])) / 10.0
#             relationship_features = [relationship_count] * 5
            
#             # Combine all features to make 90 dimensions
#             feature_vector = (
#                 [importance, emotional, temporal, access_freq] +  # 4
#                 type_encoding +  # 4
#                 content_features +  # 10
#                 temporal_features +  # 5
#                 relationship_features +  # 5
#                 [0.5] * 62  # Padding to reach 90 dimensions
#             )
            
#             # Ensure exactly 90 dimensions
#             feature_vector = feature_vector[:90]
#             while len(feature_vector) < 90:
#                 feature_vector.append(0.5)
        
#         return feature_vector
    
#     async def process_memory_through_gates(self, memory: Dict[str, Any]) -> Dict[str, Any]:
#         """Process a single memory through LSTM gates"""
        
#         # Ensure valid feature vector
#         feature_vector = self._ensure_feature_vector(memory)
        
#         # Get gate decisions
#         gate_decisions = self.gate_network.get_gate_decisions(feature_vector)
        
#         # Update memory with gate scores
#         processed_memory = memory.copy()
#         processed_memory['gate_decisions'] = gate_decisions
#         processed_memory['feature_vector'] = feature_vector
        
#         # Update importance score based on input gate
#         input_score = gate_decisions['input']['score']
#         processed_memory['importance_score'] = input_score
        
#         # Apply forget gate (memory decay)
#         forget_score = gate_decisions['forget']['score']
#         if gate_decisions['forget']['decision']:
#             # Apply decay based on forget gate
#             current_importance = processed_memory['importance_score']
#             processed_memory['importance_score'] = current_importance * (1 - forget_score)
#             self.stats['forget_gate_triggered'] += 1
        
#         # Track input gate decisions
#         if gate_decisions['input']['decision']:
#             self.stats['input_gate_accepted'] += 1
        
#         # Track output gate decisions
#         if not gate_decisions['output']['decision']:
#             self.stats['output_gate_filtered'] += 1
        
#         return processed_memory
    
#     async def filter_memories_by_gates(self, memories: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
#         """Filter memories based on gate decisions"""
        
#         filtered_memories = []
        
#         for memory in memories:
#             gate_decisions = memory.get('gate_decisions', {})
            
#             # Apply input gate filter (for new memories)
#             input_decision = gate_decisions.get('input', {}).get('decision', True)
            
#             # Apply output gate filter (for retrieval)
#             output_decision = gate_decisions.get('output', {}).get('decision', True)
            
#             # Keep memory if it passes both input and output gates
#             if input_decision and output_decision:
#                 # Additional importance threshold check
#                 importance = memory.get('importance_score', 0.0)
#                 if importance > 0.1:  # Minimum importance threshold
#                     filtered_memories.append(memory)
        
#         return filtered_memories
    
#     async def rank_memories_by_importance(self, memories: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
#         """Rank memories by importance score and relevance"""
        
#         # Sort by importance score (descending)
#         ranked_memories = sorted(
#             memories, 
#             key=lambda m: m.get('importance_score', 0.0), 
#             reverse=True
#         )
        
#         # Add ranking metadata
#         for i, memory in enumerate(ranked_memories):
#             memory['rank'] = i + 1
#             memory['percentile'] = (len(ranked_memories) - i) / len(ranked_memories)
        
#         return ranked_memories
    
#     async def process_user_memories(self, user_id: str) -> Dict[str, Any]:
#         """Complete processing pipeline for user memories"""
#         start_time = datetime.now()
        
#         print(f"\n🔄 Processing Component 5 Pipeline for user: {user_id}")
#         print("=" * 60)
        
#         try:
#             # Step 1: Retrieve memories from Astra DB
#             print("\n📊 Step 1: Retrieving memories from Astra DB...")
#             raw_memories = await self.retrieve_user_memories(user_id)
            
#             if not raw_memories:
#                 print("⚠️ No memories found for user")
#                 return {
#                     'user_id': user_id,
#                     'processed_memories': [],
#                     'statistics': self.stats,
#                     'processing_time': 0.0
#                 }
            
#             # Step 2: Process each memory through LSTM gates
#             print(f"\n🧠 Step 2: Processing {len(raw_memories)} memories through LSTM gates...")
#             processed_memories = []
            
#             for i, memory in enumerate(raw_memories):
#                 print(f"   Processing memory {i+1}/{len(raw_memories)}: {memory.get('id', 'unknown')[:8]}...")
#                 processed_memory = await self.process_memory_through_gates(memory)
#                 processed_memories.append(processed_memory)
#                 self.stats['memories_processed'] += 1
            
#             # Step 3: Filter memories based on gate decisions
#             print(f"\n🔍 Step 3: Filtering memories based on gate decisions...")
#             filtered_memories = await self.filter_memories_by_gates(processed_memories)
#             print(f"   Filtered to {len(filtered_memories)} memories")
            
#             # Step 4: Rank memories by importance
#             print(f"\n📈 Step 4: Ranking memories by importance...")
#             final_memories = await self.rank_memories_by_importance(filtered_memories)
#             self.stats['final_memories_count'] = len(final_memories)
            
#             # Calculate processing time
#             processing_time = (datetime.now() - start_time).total_seconds()
#             self.stats['processing_time'] = processing_time
            
#             # Generate summary
#             print(f"\n✅ Component 5 Processing Complete!")
#             print(f"   📊 Input memories: {len(raw_memories)}")
#             print(f"   🧠 Processed: {len(processed_memories)}")
#             print(f"   🔍 Filtered: {len(filtered_memories)}")
#             print(f"   📈 Final output: {len(final_memories)}")
#             print(f"   ⏱️ Processing time: {processing_time:.2f}s")
            
#             return {
#                 'user_id': user_id,
#                 'processed_memories': final_memories,
#                 'raw_memories': raw_memories,
#                 'gate_statistics': {
#                     'input_gate_acceptance_rate': self.stats['input_gate_accepted'] / max(1, self.stats['memories_processed']),
#                     'forget_gate_trigger_rate': self.stats['forget_gate_triggered'] / max(1, self.stats['memories_processed']),
#                     'output_gate_filter_rate': self.stats['output_gate_filtered'] / max(1, self.stats['memories_processed']),
#                     'overall_retention_rate': len(final_memories) / max(1, len(raw_memories))
#                 },
#                 'statistics': self.stats,
#                 'processing_time': processing_time
#             }
            
#         except Exception as e:
#             print(f"❌ Error in Component 5 processing: {e}")
#             import traceback
#             traceback.print_exc()
#             return {
#                 'user_id': user_id,
#                 'error': str(e),
#                 'processed_memories': [],
#                 'statistics': self.stats
#             }
    
#     def save_results(self, results: Dict[str, Any], filename: str = None):
#         """Save processing results to JSON file"""
#         if not filename:
#             timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
#             filename = f"component5_output_{results['user_id']}_{timestamp}.json"
        
#         # Convert datetime objects to strings for JSON serialization
#         serializable_results = self._make_json_serializable(results)
        
#         with open(filename, 'w') as f:
#             json.dump(serializable_results, f, indent=2)
        
#         print(f"💾 Results saved to: {filename}")
#         return filename
    
#     def _make_json_serializable(self, obj):
#         """Convert objects to JSON serializable format"""
#         if isinstance(obj, dict):
#             return {k: self._make_json_serializable(v) for k, v in obj.items()}
#         elif isinstance(obj, list):
#             return [self._make_json_serializable(item) for item in obj]
#         elif isinstance(obj, datetime):
#             return obj.isoformat()
#         elif hasattr(obj, '__dict__'):
#             return self._make_json_serializable(obj.__dict__)
#         else:
#             return obj


# async def main():
#     """Main execution function"""
#     print("🚀 Component 5 LSTM Memory Gates Pipeline")
#     print("=" * 50)
    
#     # Initialize pipeline
#     pipeline = Component5Pipeline()
#     await pipeline.initialize()
    
#     # Process memories for user_id = "123"
#     user_id = "123"
#     results = await pipeline.process_user_memories(user_id)
    
#     # Save results
#     output_file = pipeline.save_results(results)
    
#     # Display summary
#     print(f"\n📋 Processing Summary:")
#     print(f"   User ID: {results['user_id']}")
#     print(f"   Memories processed: {results['statistics']['memories_processed']}")
#     print(f"   Final memories: {results['statistics']['final_memories_count']}")
#     print(f"   Processing time: {results['statistics']['processing_time']:.2f}s")
    
#     if 'gate_statistics' in results:
#         gate_stats = results['gate_statistics']
#         print(f"\n🧠 Gate Statistics:")
#         print(f"   Input gate acceptance: {gate_stats['input_gate_acceptance_rate']:.1%}")
#         print(f"   Forget gate trigger: {gate_stats['forget_gate_trigger_rate']:.1%}")
#         print(f"   Output gate filter: {gate_stats['output_gate_filter_rate']:.1%}")
#         print(f"   Overall retention: {gate_stats['overall_retention_rate']:.1%}")
    
#     print(f"\n💾 Output saved to: {output_file}")
#     print(f"🎯 Ready for Component 6 integration!")
    
#     return results


# if __name__ == "__main__":
#     # Run the pipeline
#     results = asyncio.run(main())


NameError: name '__file__' is not defined